In [1]:
import pandas as pd
from sqlalchemy import create_engine

from service_submission_parser.src.parsers.request_parser import RequestParser
from lib_borrowbot_core.raw_objects.submission import Submission

In [2]:
%matplotlib inline

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Get Data

In [4]:
sql_parameters = {
    'host': 'localhost',
    'port': 3306,
    'user': '',
    'passwd': '',
    'db': ''
}
engine = create_engine("mysql://{}:{}@{}/{}?charset=utf8mb4".format(
    sql_parameters['user'],
    sql_parameters['passwd'],
    sql_parameters['host'],
    sql_parameters['db'],
), convert_unicode=True, encoding='utf-8')
con = engine.connect()

/home/fwang/anaconda3/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py:1569: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  cursor.execute('SELECT @@tx_isolation')


In [5]:
request_query = "SELECT * FROM requests;"
loan_query = "SELECT * FROM loans;"
confirm_query = "SELECT * FROM confirmations;"
response_query = "SELECT * FROM responses;"

submission_query = "SELECT * FROM submissions;"
comment_query = "SELECT * FROM comments;"

In [6]:
requests = pd.read_sql(request_query, con)
loans = pd.read_sql(loan_query, con)
confirms = pd.read_sql(confirm_query, con)
responses = pd.read_sql(response_query, con)

submissions = pd.read_sql(submission_query, con)
comments = pd.read_sql(comment_query, con)

In [7]:
con.close()

# Requests Data Quality

In [ ]:
# Lots of problems here with parsed semantic information

In [32]:
sample = requests [requests['principal_amt'].isnull()] \
    .merge(submissions, left_on='source_submission_id', right_on='submission_id') \
    [['principal_amt', 'title']]
    
for r in sample.iterrows():
    print(r[1]['principal_amt'])
    print(r[1]['title'])
    print()

nan
[REQ] ($100) - (#Ardmore, OK, USA) ($125 by 8/31) (Paypal)

nan
[REQ] (30$) - (#Vantaa, Uusimaa, Finland), (by 18.07 2017), (Paypal/Bitcoin)

nan
[REQ] (Amount ) - (#City, State, Country), (Repayment Date), (Payment Method/Pre-Arranged)

nan
[REQ] (20$ ) - (#New York, usa), (03/08), (PayPal)

nan
[REQ] ($1250 ) - (#Escondido, CA, USA), (Bi weekly starting 6/30), (Pre-Arranged)

nan
[REQ] VENMO 200$ PAYBACK 300$ JUNE 16TH

nan
[Req] money for school (#ottawa, ontario, Canada)

nan
[REQ] (£120/$150) - (# Sevenoaks, Kent, UK), (July 21st 2017), (Paypal/Pingit)

nan
[REQ] ($1400) - (#Silverdale, WA, USA), (6/17/16), (TBD)

nan
[REQ] (15$) - (#Vantaa, Uusimaa, Finland), (03.07.2017), (Paypal)

nan
[REQ] ($200) - (#Zagreb, Croatia), ($250 by 07/20), (Bitcoin, PayPal)

nan
[REQ] (30) - (UNITED KINGDOM) (PAYPAL)

nan
[REQ] ($2000) - (#Newburgh, NY, USA), (9/1), (Paypal)

nan
[REQ] ($1000 over 2 people?) - (#Gilbert, AZ, USA), (Monthly payments of 125), (PayPal or present post dated checks 

[REQ] ($25) - (#Sacramento, CA, USA), (6/16), (PayPal)

nan
[REQ] ($2000) (#Houston,TX,USA) (12/22/17) (PAYPAL) (Prearranged Loan)

nan
[REQ] ($30) - (New Orleans, Louisiana, USA), (Later tonight or tomorrow), (Bitcoin)

nan
[REQ] ($20) - (# Scottsdale, AZ, USA) (06/16/17) (PayPal, whatever possible)

nan
[REQ] ($10) - (#Jackson, MS, USA), (6/16/2017), (Paypal)

nan
[REQ] ($40 ) - (#United States, Las Vagas), (06/30/17), (Paypal)

nan
[REQ] (40$ USD) (#Houston, TX) (Repay 48$ 6/24) (PayPal)

nan
[REQ] ($30 USD) - (#Rural Retreat, Virginia, USA), (Repayment on 6/9/17 as $35), (Google Wallet or Simple Bank transfer)

nan
[REQ] ($25) - (Indianapolis, Indiana, USA) ($25) (REPAY $35 June 30, 2017) (paypal)

nan
[REQ] ($50) - (#Tampa, Florida, USA), ($65 on 6/16/17), (PayPal)

nan
[REQ] ($134.88) - (#Nokomis, IL, USA), (Payable over 3-4 Payments), (Payment Method - PayPal)

nan
[REQ] ($40 ) - (#Rochester, New York, USA), (June 9th, 2017), (Paypal)

nan
[REQ] (1400) - (#San Diego, CA, United 

# Pairing Quality

## Inspects the Request/Loan/Confirmation Linkage

In [ ]:
# Looks good overall,
# Suggests need to group multiple loans/confirmations under one request

In [8]:
loans_ = loans[['loan_id', 'amount', 'lender_id', 'borrower_id', 'source_submission_id', 'source_comment_id']]
loans_ = loans_.rename(columns={'borrower_id': 'loan_borrower_id', 'amount': 'loan_amount', 'lender_id': 'loan_lender_id'})

confirms_ = confirms[['confirm_id', 'amount', 'source_submission_id', 'borrower_id']]
confirms_ = confirms_.rename(columns={'amount': 'confirm_amount', 'borrower_id': 'confirm_borrower_id'})

join_ = requests[['request_id', 'borrower_id', 'principal_amt', 'repay_amt', 'source_submission_id']] \
    .merge(loans_, how="left", on="source_submission_id") \
    .merge(confirms_, how='left', on="source_submission_id") \
    .merge(submissions, how='left', left_on='source_submission_id', right_on='submission_id')
    
join_ = join_[['request_id', 'source_submission_id', 'num_comments', 'loan_id', 'confirm_id', 'principal_amt', 'repay_amt', 'loan_amount', 'confirm_amount', 'borrower_id', 'loan_borrower_id', 'confirm_borrower_id']]

In [16]:
join_[join_.duplicated('request_id')]

,request_id,source_submission_id,num_comments,loan_id,confirm_id,principal_amt,repay_amt,loan_amount,confirm_amount,borrower_id,loan_borrower_id,confirm_borrower_id
40,req_007c2117-9d29-4171-9dd3-07cbe40970d7,t3_8d9heo,9,loan_43677624-38eb-4253-8669-98b049064e52,confirm_85ccc901-abce-4392-9179-9eb9240bc9e2,250.0,305.0,250.0,NaN,t2_c4ou2,t2_c4ou2,t2_c4ou2
41,req_007c2117-9d29-4171-9dd3-07cbe40970d7,t3_8d9heo,9,loan_c1120478-1c58-4530-ae52-f4494dfad842,confirm_67f00e9c-ffd7-421f-a328-333f7406d645,250.0,305.0,50.0,NaN,t2_c4ou2,t2_c4ou2,t2_c4ou2
42,req_007c2117-9d29-4171-9dd3-07cbe40970d7,t3_8d9heo,9,loan_c1120478-1c58-4530-ae52-f4494dfad842,confirm_85ccc901-abce-4392-9179-9eb9240bc9e2,250.0,305.0,50.0,NaN,t2_c4ou2,t2_c4ou2,t2_c4ou2
61,req_00aba23d-cd54-4c64-9d4c-27ec570e1faa,t3_7hveq5,10,loan_111be534-2b60-41a8-a44d-f140399f8bd0,confirm_c93bf082-32fe-4340-bfd1-9acbbb080f6e,800.0,NaN,800.0,NaN,t2_8g83t,t2_8g83t,t2_8g83t
62,req_00aba23d-cd54-4c64-9d4c-27ec570e1faa,t3_7hveq5,10,loan_91d06673-0c31-4297-813f-71b6cb972540,confirm_390ae1c3-78bc-45e2-8332-0686ef99f36c,800.0,NaN,150.0,NaN,t2_8g83t,t2_8g83t,t2_8g83t
63,req_00aba23d-cd54-4c64-9d4c-27ec570e1faa,t3_7hveq5,10,loan_91d06673-0c31-4297-813f-71b6cb972540,confirm_c93bf082-32fe-4340-bfd1-9acbbb080f6e,800.0,NaN,150.0,NaN,t2_8g83t,t2_8g83t,t2_8g83t
91,req_00ef99a0-11d5-4b4f-a8f6-c059d6bd30c1,t3_6ffkqe,8,loan_b13f6507-84e1-489c-881b-1baffff7ac4d,NaN,870.0,NaN,600.0,NaN,t2_sec8q,t2_sec8q,NaN
101,req_01056344-7f20-4b41-8b72-c028b56e22f9,t3_77o65z,73,loan_0d0b3937-35f7-48c9-bd0c-6e99116825dc,confirm_5ed6af86-f24e-4df6-8e12-10a4dac5dc6f,160.0,NaN,860.0,NaN,t2_3qdp9,t2_3qdp9,t2_3qdp9
102,req_01056344-7f20-4b41-8b72-c028b56e22f9,t3_77o65z,73,loan_0d0b3937-35f7-48c9-bd0c-6e99116825dc,confirm_64852d7e-54e6-4b4b-aaf2-ed14845a3347,160.0,NaN,860.0,NaN,t2_3qdp9,t2_3qdp9,t2_3qdp9
103,req_01056344-7f20-4b41-8b72-c028b56e22f9,t3_77o65z,73,loan_0d0b3937-35f7-48c9-bd0c-6e99116825dc,confirm_6becb54a-fb8f-47b8-b675-914444b4dbfe,160.0,NaN,860.0,NaN,t2_3qdp9,t2_3qdp9,t2_3qdp9


## Inspects Response Linkage

In [ ]:
# source_submission_id in responses seems wrong

In [ ]:
# probably better to link by user accounts than by request sequence
# requests are too loosely followed

In [28]:
user_lookups = pd.concat([submissions[['author_name', 'author_id']], comments[['author_name', 'author_id']]])
user_lookups = user_lookups.groupby('author_id').first().reset_index()

In [37]:
responses_ = responses[['response_id', 'amount', 'repaid', 'lender_id', 'borrower_name']].merge(user_lookups, how='left', left_on='borrower_name', right_on='author_name')
responses_ = responses_.rename({'author_id': 'borrower_id'})[['response_id', 'amount', 'repaid', 'lender_id', 'author_id']]

In [41]:
loans_ = loans[['loan_id', 'amount', 'lender_id', 'borrower_id', 'source_submission_id', 'source_comment_id']]
loans_ = loans_.rename(columns={'borrower_id': 'loan_borrower_id', 'amount': 'loan_amount', 'lender_id': 'loan_lender_id'})

join_ = requests[['request_id', 'borrower_id', 'principal_amt', 'repay_amt', 'source_submission_id']] \
    .merge(loans_, how="left", on="source_submission_id") \
#     .merge(responses_, how='left', on="source_submission_id") \
#     .merge(submissions, how='left', left_on='source_submission_id', right_on='submission_id')
    
# join_ = join_[['request_id', 'source_submission_id', 'num_comments', 'loan_id', 'confirm_id', 'principal_amt', 'repay_amt', 'loan_amount', 'confirm_amount', 'borrower_id', 'loan_borrower_id', 'confirm_borrower_id']]